### Virtual Wirewalkers

In [4]:
import netCDF4 as nc
import matplotlib.pyplot as plt
import numpy as np
import seawater as sw
import cmocean.cm as cm
import datetime 
import pandas as pd

import glob
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
from matplotlib.gridspec import GridSpec
from matplotlib import rc  

dat = nc.Dataset('/Users/michal/Data/cmems_mod_glo_phy-cur_anfc_0.083deg_PT6H-i_1730570948231.nc')

time=dat['time'][:]

day1=datetime.datetime(1970,1,1)+datetime.timedelta(seconds=int(time[0]))
timev=pd.date_range(day1,periods=time.size,freq='6H')

lat=dat['latitude'][:]
lon=dat['longitude'][:]

RLON,RLAT=np.meshgrid(lon,lat)

U=dat['uo'][:][:,:]
V=dat['vo'][:][:,:]

dep=-dat['depth'][:].data

from scipy import integrate
U=integrate.trapezoid(U,dep,axis=1)/dep[-1]
V=integrate.trapezoid(V,dep,axis=1)/dep[-1]

U=U[14*4:]
V=V[14*4:]


ModuleNotFoundError: No module named 'matplotlib'

In [4]:
X = RLAT.copy()*np.nan
Y = RLAT.copy()*np.nan
for i in range(RLAT.shape[0]):
    X[i] = np.cumsum(np.append(0, sw.dist(RLAT[i],RLON[i], units='km')[0]))*1000
for i in range(RLAT.shape[1]):
    Y[:,i] = np.cumsum(np.append(0, sw.dist(RLAT[:,i],RLON[:,i], units='km')[0]))*1000


# write netCDF file
#------------------

# open a netCDF file to write
ncout = nc.Dataset('/Users/caiquediasluko/Documents/Caique/MOTIVE/Wirewalker_Forecast/Glorystraj2_0-760m_U.nc', 'w', format='NETCDF4')

# define axis size
ncout.createDimension('time_counter', U.shape[0])  # unlimited
ncout.createDimension('x', X[0].shape[0])
ncout.createDimension('y', Y[:,0].shape[0])
ncout.createDimension('depthu', 1)

from numpy import dtype

y = ncout.createVariable('y', dtype('float64').char, ('y',))
y.axis = 'y'
x = ncout.createVariable('x', dtype('float64').char, ('x',))
x.axis = 'x'
vozocrtx = ncout.createVariable('vozocrtx', dtype('float32').char, ('time_counter','depthu','y','x'))
depthu = ncout.createVariable('depthu', dtype('float32').char, ('depthu',))
depthu.axis = 'depthu'
nav_lon = ncout.createVariable('nav_lon', dtype('float32').char, ('y','x'))
time_counter = ncout.createVariable('time_counter', dtype('float64').char, ('time_counter',))
time_counter.axis = 'time_counter'
nav_lat = ncout.createVariable('nav_lat', dtype('float32').char, ('y','x'))

# copy axis from original dataset
t = np.arange(U.shape[0])*6*3600
time_counter[:] = t[:]

x[:] = X[0][:]
y[:] = Y[:,0][:]
nav_lat[:] = RLAT[:]
nav_lon[:] = RLON[:]
depthu[:] = np.array([0])[:]
vozocrtx[:] = U[:]

# close files
ncout.close()

In [5]:
# open a netCDF file to write
ncout = nc.Dataset('/Users/caiquediasluko/Documents/Caique/MOTIVE/Wirewalker_Forecast/Glorystraj2_0-760m_V.nc', 'w', format='NETCDF4')
from numpy import dtype

# define axis size
ncout.createDimension('time_counter', V.shape[0])  # unlimited
ncout.createDimension('x', X[0].shape[0])
ncout.createDimension('y', Y[:,0].shape[0])
ncout.createDimension('depthv', 1)

y = ncout.createVariable('y', dtype('float64').char, ('y',))
y.axis = 'y'
x = ncout.createVariable('x', dtype('float64').char, ('x',))
x.axis = 'x'
vomecrty = ncout.createVariable('vomecrty', dtype('float32').char, ('time_counter','depthv','y','x'))
depthv = ncout.createVariable('depthv', dtype('float32').char, ('depthv',))
depthv.axis = 'depthv'
nav_lon = ncout.createVariable('nav_lon', dtype('float32').char, ('y','x'))
time_counter = ncout.createVariable('time_counter', dtype('float64').char, ('time_counter',))
time_counter.axis = 'time_counter'
nav_lat = ncout.createVariable('nav_lat', dtype('float32').char, ('y','x'))

# copy axis from original dataset
t = np.arange(U.shape[0])*6*3600
time_counter[:] = t[:]

x[:] = X[0][:]
y[:] = Y[:,0][:]
nav_lat[:] = RLAT[:]
nav_lon[:] = RLON[:]
depthv[:] = np.array([0])[:]
vomecrty[:] = V[:]

# close files
ncout.close()
